In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
import evaluate

import pandas as pd

df_train = pd.read_csv('/home/aflah20082/NLP_Project/Data/PreprocessedData/train_preprocessed.csv')
df_test = pd.read_csv('/home/aflah20082/NLP_Project/Data/PreprocessedData/test_preprocessed.csv')
df_val = pd.read_csv('/home/aflah20082/NLP_Project/Data/PreprocessedData/val_preprocessed.csv')

# Add Dummy Labels to Test
df_test['label'] = 0

df_train = df_train[['preprocessed_text', 'label']]
df_test = df_test[['preprocessed_text', 'label']]
df_val = df_val[['preprocessed_text', 'label']]

In [ ]:
df_train = df_train.rename(columns={'preprocessed_text': 'text', 'label': 'label'})
df_test = df_test.rename(columns={'preprocessed_text': 'text', 'label': 'label'})
df_val = df_val.rename(columns={'preprocessed_text': 'text', 'label': 'label'})

In [ ]:
# Change Label to 0 and 1
df_train['label'] = df_train['label'].replace({'NOT': 1, 'OFF': 0})
df_val['label'] = df_val['label'].replace({'NOT': 1, 'OFF': 0})

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import AutoModelForSequenceClassification

bertweet_tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", force_download=True)
bertbase_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", force_download=True)
hatebert_tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT", force_download=True)

In [ ]:
from datasets import load_dataset,Dataset,DatasetDict
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AutoModel, AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd

dataset_hf = DatasetDict({
    'train': Dataset.from_pandas(df_train),
    'test': Dataset.from_pandas(df_test),
    'validation': Dataset.from_pandas(df_val),
    'train_val': Dataset.from_pandas(pd.concat([df_train, df_val]))
})

bertweet_tokenized_dataset = dataset_hf.map(lambda examples: bertweet_tokenizer(examples['text'], truncation=True, padding='max_length'), batched=True, batch_size=16)
bertbase_tokenized_dataset = dataset_hf.map(lambda examples: bertbase_tokenizer(examples['text'], truncation=True, padding='max_length'), batched=True, batch_size=16)
hatebert_tokenized_dataset = dataset_hf.map(lambda examples: hatebert_tokenizer(examples['text'], truncation=True, padding='max_length'), batched=True, batch_size=16)

In [ ]:
# Combine the two datasets
dataset_hf = DatasetDict({
    'train': Dataset.from_pandas(df_train),
    'test': Dataset.from_pandas(df_test),
    'validation': Dataset.from_pandas(df_val),
    'train_val': Dataset.from_pandas(pd.concat([df_train, df_val]))
})

bertweet_tokenized_dataset = dataset_hf.map(lambda examples: bertweet_tokenizer(examples['text'], truncation=True, padding='max_length'), batched=True)
bertbase_tokenized_dataset = dataset_hf.map(lambda examples: bertbase_tokenizer(examples['text'], truncation=True, padding='max_length'), batched=True)

bertweet_tokenized_dataset.set_format("torch",columns=["input_ids", "attention_mask", "label"])
data_collator_bertweet = DataCollatorWithPadding(tokenizer=bertweet_tokenizer)

bertbase_tokenized_dataset.set_format("torch",columns=["input_ids", "attention_mask", "label"])
data_collator_bertbase = DataCollatorWithPadding(tokenizer=bertbase_tokenizer)

hatebert_tokenized_dataset.set_format("torch",columns=["input_ids", "attention_mask", "label"])
data_collator_hatebert = DataCollatorWithPadding(tokenizer=hatebert_tokenizer)

# Combine the two datasets
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, bertweet_dataset, bertbase_dataset, hatebert_dataset):
        self.dataset1 = bertweet_dataset
        self.dataset2 = bertbase_dataset
        self.dataset3 = hatebert_dataset

    def __getitem__(self, index):
        return {
            'input_ids_bertweet': self.dataset1[index]['input_ids'],
            'attention_mask_bertweet': self.dataset1[index]['attention_mask'],
            'input_ids_bertbase': self.dataset2[index]['input_ids'],
            'attention_mask_bertbase': self.dataset2[index]['attention_mask'],
            'input_ids_hatebert': self.dataset3[index]['input_ids'],
            'attention_mask_hatebert': self.dataset3[index]['attention_mask'],
            'label': self.dataset1[index]['label']
        }

    def __len__(self):
        return len(self.dataset1)

train_dataset = ConcatDataset(bertweet_tokenized_dataset['train'], bertbase_tokenized_dataset['train'], hatebert_tokenized_dataset['train'])
test_dataset = ConcatDataset(bertweet_tokenized_dataset['test'], bertbase_tokenized_dataset['test'], hatebert_tokenized_dataset['test'])
val_dataset = ConcatDataset(bertweet_tokenized_dataset['validation'], bertbase_tokenized_dataset['validation'], hatebert_tokenized_dataset['validation'])
train_val_dataset = ConcatDataset(bertweet_tokenized_dataset['train_val'], bertbase_tokenized_dataset['train_val'], hatebert_tokenized_dataset['train_val'])

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=16)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=16)
train_val_dataloader = DataLoader(train_val_dataset, shuffle=True, batch_size=16)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
from torch import nn
class ConcatModel(torch.nn.Module):
    def __init__(self):
        super(ConcatModel, self).__init__()
        self.bertweet_model = AutoModel.from_pretrained("vinai/bertweet-base", config = AutoConfig.from_pretrained("vinai/bertweet-base", 
                                                                                                       output_attention = True, 
                                                                                                       output_hidden_state = True ) )
                                            
        self.bertbase_model = AutoModel.from_pretrained("bert-base-uncased", config = AutoConfig.from_pretrained("bert-base-uncased",
                                                                                                         output_attention = True, 
                                                                                                         output_hidden_state = True ) )  

        self.hatebert_model = AutoModel.from_pretrained("GroNLP/hateBERT", config = AutoConfig.from_pretrained("GroNLP/hateBERT",
                                                                                                         output_attention = True, 
                                                                                                         output_hidden_state = True ) )
        # Freeze first 10 layers of bertweet
        for param in self.bertweet_model.base_model.encoder.layer[:8].parameters():
            param.requires_grad = False
        
        # Freeze first 10 layers of bertbase
        for param in self.bertbase_model.base_model.encoder.layer[:8].parameters():
            param.requires_grad = False
            
        # Freeze first 10 layers of hatebert
        for param in self.hatebert_model.base_model.encoder.layer[:8].parameters():
            param.requires_grad = False

        self.classifier = nn.Sequential(
            # nn.Dropout(p=0.1),
            nn.Linear(768*3, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 2)
            # nn.Softmax(dim=1)
        )
    
    def forward(self, input_ids_bertweet, attention_mask_bertweet, input_ids_bertbase, attention_mask_bertbase, input_ids_hatebert, attention_mask_hatebert, label=None):
        logits_a = self.bertweet_model(input_ids_bertweet, attention_mask=attention_mask_bertweet).last_hidden_state[:, 0, :].view(-1, 768)
        logits_b = self.bertbase_model(input_ids_bertbase, attention_mask=attention_mask_bertbase).last_hidden_state[:, 0, :].view(-1, 768)
        logits_c = self.hatebert_model(input_ids_hatebert, attention_mask=attention_mask_hatebert).last_hidden_state[:, 0, :].view(-1, 768)
        # summed_vectors = torch.mean(torch.stack([logits_a, logits_b, logits_c]), dim=0)
        concat_vectors = torch.cat((logits_a, logits_b, logits_c), dim=1)
        output = self.classifier(concat_vectors)
        # print(output.shape)
        # Compute Loss
        loss = None
        if label is not None:
            loss_fct = nn.CrossEntropyLoss()
            # print(output.shape)
            # print(label.shape)
            # print(label.view(-1).shape)
            # print(output.view(-1, 2).shape)
            loss = loss_fct(output.view(-1, 2), label.view(-1))
            
            return TokenClassifierOutput(loss=loss, logits=output, hidden_states=None, attentions=None)

        
model = ConcatModel().to(device)

In [ ]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters())

num_epoch = 2

num_training_steps = num_epoch * len(train_dataloader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer = optimizer,
    num_warmup_steps = 10000,
    num_training_steps = num_training_steps,
    
)

In [ ]:
from datasets import load_metric
metric = load_metric("accuracy")
f1 = load_metric("f1")

In [ ]:
# from tqdm.auto import tqdm
# num_training_steps = num_epoch * len(train_dataloader)
# warmup_steps = 0
# num_epochs = 2

# progress_bar_train = tqdm(range(num_training_steps))
# progress_bar_eval = tqdm(range(num_epochs * len(val_dataloader)))

# step = 0

# for epoch in range(num_epochs):
  
#   for batch in train_dataloader:
#       model.train()
#       # print([type(v) for v in batch.values()])
#       batch = {k: v.to(device) for k, v in batch.items()}
      
#       outputs = model(**batch)
#       loss = outputs.loss
#       loss.backward()

#       optimizer.step()
#       lr_scheduler.step()
#       optimizer.zero_grad()
#       progress_bar_train.update(1)
#       step += 1

#       if step % 100 == 0:
#           print(f"Step {step} of {num_training_steps}: loss = {loss.item()}")
#           # Save model
#           torch.save(model.state_dict(), f"/home/aflah20082/NLP_Project/Models/CustomModelSaves/model_{step}.pt")
          

#   model.eval()
#   for batch in val_dataloader:
#     # print(batch.keys())
#     batch = {k: v.to(device) for k, v in batch.items()}
#     with torch.no_grad():
#         outputs = model(**batch)

#     logits = outputs.logits
#     predictions = torch.argmax(logits, dim=-1)
#     metric.add_batch(predictions=predictions, references=batch["label"])
#     progress_bar_eval.update(1)
    
#   print(metric.compute())

In [ ]:
# Save the model
# torch.save(model.state_dict(), "model.pt")

# Load the model
model2 = ConcatModel().to(device)
model2.load_state_dict(torch.load("/home/aflah20082/NLP_Project/Models/CustomModelSaves/model_600.pt"))

In [ ]:
model2.eval()

for batch in val_dataloader:
    # print(batch.keys())
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model2(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["label"])
    f1.add_batch(predictions=predictions, references=batch["label"])

print(metric.compute(), f1.compute(average="macro"))

In [ ]:
model2.eval()

for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model2(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

In [ ]:
# pickle the model
import pickle
with open('firstconcatmodel.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# load the model
import pickle
with open('firstconcatmodel.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
model.eval()
ls_preds = []
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    ls_preds.append(predictions)



In [ ]:
ls_preds